In [34]:
import os
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [35]:
PATH_ROOT = Path(os.getcwd()).parent.absolute()
PATH_RAW = os.path.join(PATH_ROOT, "data", "raw_data")
PATH_CLEAN = os.path.join(PATH_ROOT, "data", "clean_data")

In [36]:
def fill_nan_with_mean_from_prev_and_next(df):
    # errors whould occure if there is no previous available value or next available value
        
        count = 0
        NANrows = pd.isnull(df).any(1).to_numpy().nonzero()[0]
        null_df = df.isnull()
        
        for row in NANrows :
            
            for column in range(0, df.shape[1]):

                    prev_av = None # previous_available_value
                    prev_i = 1
                    while pd.isna(prev_av):
                        prev_av = df.iat[row - prev_i, column]
                        prev_i += 1
                    
                    next_av = None # next_available_value
                    next_i = 1
                    while pd.isna(next_av):
                        next_av = df.iat[row + next_i, column]
                        next_i += 1
                        
                    df.iat[row, column] = (float(prev_av) + float(next_av)) / 2

                    #print(df.iat[row, column], prev_av, next_av, (prev_av + next_av) / 2)
                    count += 1 
                    
        print("There are " + str(count) + " Missing values filled.")

        return df

### Germany Electricity Consumption (Electricity)

In [37]:
df1 = pd.read_csv(os.path.join(PATH_RAW, "opsd_germany_daily.csv"), sep = ",")
df1 = df1.iloc[2191:, 1:-1].copy()
df1 = df1.reset_index(drop = True)
print(df1.shape)
df1.head(5)

(2192, 3)


,Consumption,Wind,Solar
0,948.128,227.465,6.587
1,1269.581,207.327,6.574
2,1334.745,473.468,24.679
3,1347.136,499.804,14.681
4,1376.658,523.851,5.071


In [38]:
df1.isnull().values.sum()

5

In [39]:
df1 = fill_nan_with_mean_from_prev_and_next(df1)
df1.isnull().values.sum()

There are 15 Missing values filled.


0

In [40]:
df1.to_csv(os.path.join(PATH_CLEAN, "Electricity.csv"), index = False)

### New Zealand Land Temperatures d02

In [41]:
df2 = pd.read_csv(os.path.join(PATH_RAW, "GlobalLandTemperaturesByCity.csv"), sep = ",")
df2 = df2.loc[df2["Country"] == "New Zealand"]
df2 = df2.reset_index(drop = True)
df2 = df2.iloc[:, :-3]
df2.head(5)

,dt,AverageTemperature,AverageTemperatureUncertainty,City
0,1853-01-01,17.401,1.969,Auckland
1,1853-02-01,17.355,1.806,Auckland
2,1853-03-01,16.740,2.013,Auckland
3,1853-04-01,15.230,1.754,Auckland
4,1853-05-01,13.792,1.440,Auckland


In [42]:
df2 = df2.pivot(index = "dt", columns = "City", values = "AverageTemperature")
df2 = df2.iloc[144:-1,:]
df2

City,Auckland,Christchurch,Dunedin,Hamilton,Lower Hutt,North Shore,Tauranga,Waitakere,Wellington
dt,,,,,,,,,
1853-01-01,17.401,11.954,11.075,17.157,14.789,17.401,16.218,17.401,14.789
1853-02-01,17.355,11.613,10.524,16.905,14.516,17.355,15.869,17.355,14.516
1853-03-01,16.740,10.519,8.526,15.640,13.542,16.740,14.897,16.740,13.542
1853-04-01,15.230,9.248,6.036,13.423,12.088,15.230,13.170,15.230,12.088
1853-05-01,13.792,7.627,3.035,11.307,10.593,13.792,11.580,13.792,10.593
...,...,...,...,...,...,...,...,...,...
2013-04-01,17.839,11.941,8.543,16.224,15.112,17.839,16.037,17.839,15.112
2013-05-01,15.278,9.501,5.216,12.771,12.102,15.278,13.028,15.278,12.102
2013-06-01,13.450,7.129,1.919,10.517,10.164,13.450,11.150,13.450,10.164


In [43]:
df2.isnull().values.sum()

0

In [44]:
df2.to_csv(os.path.join(PATH_CLEAN, "NZTemp.csv"), index = False)

## Yahoo Finance
### CNY USD Exchange Rate d03

In [45]:
import yfinance as yf

In [46]:
df3 = yf.download("CNY=X", start="2018-01-01", end="2022-01-01")
df3.isnull().values.sum()

[*********************100%***********************]  1 of 1 completed


0

In [47]:
df3 = df3.iloc[:,:-1] # Exclude Volume
df3.to_csv(os.path.join(PATH_CLEAN, "CNYExch.csv"), index = False)
df3

,Open,High,Low,Close,Adj Close
Date,,,,,
2018-01-01,6.5057,6.5064,6.5010,6.5057,6.5057
2018-01-02,6.5058,6.5073,6.4801,6.5058,6.5058
2018-01-03,6.4926,6.5093,6.4900,6.4940,6.4940
2018-01-04,6.5028,6.5089,6.4870,6.5031,6.5031
2018-01-05,6.4932,6.4971,6.4694,6.4933,6.4933
...,...,...,...,...,...
2021-12-27,6.3666,6.3722,6.3613,6.3666,6.3666
2021-12-28,6.3698,6.3715,6.3608,6.3698,6.3698
2021-12-29,6.3672,6.3714,6.3584,6.3672,6.3672


### Oil d04

In [48]:
df4 = yf.download("OIL", start="2018-01-01", end="2022-01-01")
df4.isnull().values.sum()

[*********************100%***********************]  1 of 1 completed


0

In [49]:
df4.to_csv(os.path.join(PATH_CLEAN, "Oil.csv"), index = False)
df4

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,17.709999,17.709999,17.709999,17.709999,17.709999,0
2018-01-03,17.980000,18.070000,17.980000,18.070000,18.070000,1800
2018-01-04,18.040001,18.040001,18.040001,18.040001,18.040001,100
2018-01-05,18.030001,18.030001,18.030001,18.030001,18.030001,400
2018-01-08,18.030001,18.030001,18.030001,18.030001,18.030001,0
...,...,...,...,...,...,...
2021-12-23,22.610001,22.980000,22.590000,22.950001,22.950001,46800
2021-12-27,22.870001,23.549999,22.820000,23.549999,23.549999,32400
2021-12-28,23.680000,23.760000,23.520000,23.540001,23.540001,93400
